In [ ]:
import time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
calendar_data = pd.read_csv('Data/calendar.csv')
sales_data = pd.read_csv('Data/sales_train_validation.csv')
weekly_sale_price_data = pd.read_csv('Data/sell_prices.csv')

In [ ]:
calendar_data['date'] = pd.DatetimeIndex(calendar_data.date)

In [ ]:
sales_data_long = pd.melt(sales_data, 
                          id_vars = sales_data.columns[:6], value_vars = sales_data.columns[6:], 
                          var_name = 'd', value_name = 'sales')

In [ ]:
print(sales_data_long.shape)
sales_data_long.head()

In [ ]:
complete_data = sales_data_long.join(calendar_data.set_index('d'), 
                                     on = ['d'], how='left')
complete_data = complete_data.join(weekly_sale_price_data.set_index(['store_id', 'item_id', 'wm_yr_wk']), 
                                   on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')

In [ ]:
print(complete_data.shape)
print(complete_data.columns)
complete_data.head()

In [ ]:
complete_data['d'] = complete_data.d.apply(lambda x: x.split('_')[1]).astype(int)
complete_data['event_name_1'].fillna('', inplace = True)
complete_data['event_type_1'].fillna('', inplace = True)
complete_data['event_day'] = complete_data['event_name_1'].apply(lambda x: 0 if x == '' else 1)

In [ ]:
d_max = complete_data['d'].max()
complete_data_2 = complete_data[complete_data['d'] >= (d_max-365)]
complete_data_2.shape

In [ ]:
complete_data_2.head()

In [ ]:
series_id = 'FOODS_1_011_CA_1_validation'
    
fig, axs = plt.subplots(nrows = 2, ncols = 2, figsize = (16, 8), sharex = True)
sns.lineplot(x = "date", y = "sales", 
             data = complete_data_2[complete_data_2['id'] == series_id], 
             ax = axs[0, 0]).set_title('Sales')
sns.lineplot(x = "date", y = "sell_price", 
             data = complete_data_2[complete_data_2['id'] == series_id], 
             ax = axs[0, 1]).set_title('Weekly Price')
sns.lineplot(x = "date", y = "event_day", # hue="event_type_1", 
             data = complete_data_2[complete_data_2['id'] == series_id], 
             ax = axs[1, 0]).set_title('Holidays')
sns.lineplot(x = "date", y = "sell_price", 
             data = complete_data_2[complete_data_2['id'] == series_id], 
             ax = axs[1, 1]).set_title('Weekly Price')

for ax in axs.flat:
    for label in ax.get_xticklabels():
        label.set_rotation(45)

In [ ]:
series_id = 'FOODS_1_021_CA_1_validation'

In [ ]:
series_id = 'FOODS_1_031_CA_1_validation'

In [ ]:
series_id = 'HOBBIES_1_011_CA_1_validation'

In [ ]:
series_id = 'HOBBIES_1_021_CA_1_validation'

In [ ]:
series_id = 'HOBBIES_1_031_CA_1_validation'

In [ ]:
series_id = 'HOUSEHOLD_1_011_CA_1_validation'

In [ ]:
series_id = 'HOUSEHOLD_1_021_CA_1_validation'

In [ ]:
series_id = 'HOUSEHOLD_1_032_CA_1_validation'